# Descrição do problema

No âmbito da unidade curricular de Ciência de Dados, foi proposto o desenvolvimento de um modelo de machine learning utilizando o dataset "New York City Taxi Trips". O objetivo é construir um modelo capaz de prever o preço das viagens de táxi com base em diversas variáveis disponíveis, como distância da viagem, horário do dia, número de passageiros e possíveis condições de tráfego.

# Objetivo
O principal objetivo deste projeto é desenvolver um modelo que consiga estimar o valor das tarifas de táxi em Nova York. A tarefa será abordada sob duas perspectivas:

1. Regressão: Criar um modelo que consiga prever com precisão o valor exato da tarifa para uma determinada viagem.
2. Classificação: Reformular o problema para classificar as viagens em faixas de preços predefinidas:

    - Classe 1: Viagens curtas e de baixo custo (`< $10`)

    - Classe 2: Viagens de média distância e preço moderado (`$10 - $30`)

    - Classe 3: Viagens longas com tarifas mais elevadas (`$30 - $60`)

    - Classe 4: Tarifas premium (`> $60`)

# Descrição dos Dados

O dataset **New York City Yellow Taxi Trip Records** contém informações detalhadas sobre viagens de táxi na cidade de Nova York, incluindo dados sobre tempo, distância, localizações de embarque e desembarque, além de informações de pagamento.

O dataset inclui as seguintes colunas:

- **VendorID**: Código que identifica o provedor do sistema de processamento eletrônico de pagamentos (TPEP).
  - 1 = Creative Mobile Technologies, LLC
  - 2 = VeriFone Inc.

- **tpep_pickup_datetime**: Data e hora em que o taxímetro foi ativado.
- **tpep_dropoff_datetime**: Data e hora em que o taxímetro foi desativado.
- **Passenger_count**: Número de passageiros no veículo (valor inserido pelo motorista).
- **Trip_distance**: Distância percorrida na viagem (em milhas) reportada pelo taxímetro.
- **PULocationID**: Código da zona TLC onde o taxímetro foi ativado (embarque).
- **DOLocationID**: Código da zona TLC onde o taxímetro foi desativado (desembarque).
- **RateCodeID**: Código da tarifa aplicada ao final da viagem.
  - 1 = Tarifa padrão
  - 2 = JFK
  - 3 = Newark
  - 4 = Nassau ou Westchester
  - 5 = Tarifa negociada
  - 6 = Viagem em grupo

- **Store_and_fwd_flag**: Indica se o registro da viagem foi armazenado antes de ser enviado ao provedor, devido à falta de conexão do veículo com o servidor.
  - Y = Viagem armazenada antes do envio
  - N = Viagem enviada em tempo real

- **Payment_type**: Código que indica a forma de pagamento utilizada pelo passageiro.
  - 1 = Cartão de crédito
  - 2 = Dinheiro
  - 3 = Sem cobrança
  - 4 = Disputa
  - 5 = Desconhecido
  - 6 = Viagem cancelada

- **Fare_amount**: Valor da tarifa baseado no tempo e na distância percorrida.
- **Extra**: Cobranças adicionais, como sobretaxas noturnas ou de horário de pico.
- **MTA_tax**: Taxa de `$0,50` do MTA aplicada automaticamente com base na tarifa do taxímetro.
- **Improvement_surcharge**: Taxa de melhoria de `$0,30` aplicada desde 2015.
- **Tip_amount**: Valor da gorjeta (preenchido automaticamente para pagamentos com cartão; gorjetas em dinheiro não são registradas).
- **Tolls_amount**: Valor total de pedágios pagos durante a viagem.
- **Total_amount**: Valor total cobrado do passageiro (não inclui gorjetas pagas em dinheiro).


In [ ]:
# Libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3 as sql
import kagglehub

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler


# Data Analysis

A análise de dados será realizada com a abordagem ELT (Extract, Load, Transform). Inicialmente, os dados serão extraídos do KaggleHub e uma amostra aleatória será utilizada para trabalhar com uma fração representativa do conjunto completo. Em seguida, os dados extraídos serão carregados em estruturas como DataFrames Pandas ou arrays NumPy, facilitando a manipulação e análise. Por fim, na fase de transformação, os dados serão limpos, normalizados e preparados para gerar métricas e insights. A amostragem aleatória permite reduzir a complexidade, permitindo uma análise eficiente sem processar todos os dados de uma vez.

In [ ]:
class KagglehubDatabaseLoader:
    """
    A class to download and manage datasets from KaggleHub.

    This class automates the process of downloading a dataset from KaggleHub
    and provides a method to retrieve the local file path.

    Attributes:
        _dataset (str): The name of the dataset to be downloaded.
        _path (str or None): The local directory path where the dataset is saved.
    """

    def __init__(self, dataset: str):
        """
        Initializes the KagglehubDatabaseLoader class.

        Args:
            dataset (str): The name of the dataset to be downloaded.
        """
        self._dataset = dataset
        self._path = None

        self._download_dataset()


    def _download_dataset(self) -> None:
        """
        Downloads the specified dataset from KaggleHub.

        This method attempts to download the dataset and assigns the local
        file path to the `_path` attribute. If an error occurs during the
        download, an exception is caught and printed.
        """
        try:
            self._path = kagglehub.dataset_download(self._dataset)
            print("Dataset downloaded Successfully ")
            print("Dataset saved on: ", self._path)
        except Exception as e:
            print("Error downloading dataset:", e)


    def get_path(self) -> str | None:
        """
        Returns the local directory path of the downloaded dataset.

        Returns:
            str or None: The file path of the dataset if the download was
                        successful, otherwise None.
        """
        return self._path


class KagglehubSQLiteLoader(KagglehubDatabaseLoader):
    """
    A subclass of KagglehubDatabaseLoader to interact with an SQLite dataset.

    This class adds methods to retrieve information about tables, numpy arrays and dataframes.
    the SQLite dataset is downloaded from KaggleHub.

    Attributes:
        _dataset (str): The name of the dataset to be downloaded from KaggleHub.
        _path (str or None): The local directory path where the dataset is saved.
        _file_path (str or None): The complete path to the specific SQLite file within the dataset.
        _conn: The SQLite connection object used to interact with the database.
        _cursor: The cursor object for executing SQL queries on the database.
    """

    def __init__(self, dataset: str, file: str):
        """
        Initializes the KagglehubSQLLoader class by calling the superclass constructor
        to download the dataset and setting up the SQLite connection.

        Args:
            dataset (str): The dataset name to be downloaded.
            file (str): The name of the SQLite file in the downloaded dataset to be opened.
        """
        super().__init__(dataset)
        self._file_path = self._path + file
        self._conn = sql.connect(self._file_path)
        self._cursor = self._conn.cursor()


    def get_table_names(self) -> list | None:
        """
        Retrieves all table names from the SQLite database.

        Returns:
            list: A list of table names if successful, None if an error occurs.
        """
        try:
            table_name_query = "SELECT name FROM sqlite_master WHERE type='table';"
            return self._cursor.execute(table_name_query).fetchall()
        except Exception as e:
            print("Error getting table names:", e)


    def get_column_names(self, table: str) -> list | None:
        """
        Retrieves the column names for a given table in the SQLite database.

        Args:
            table (str): The name of the table for which to get column names.

        Returns:
            list: A list of column names if successful, None if an error occurs.
        """
        try:
            self._cursor.execute(f"PRAGMA table_info({table});")
            columns = [col[1] for col in self._cursor.fetchall()]
            return columns
        except Exception as e:
            print("Error getting column names:", e)


    def get_table_row_count(self, table: str) -> int | None:
        """
        Retrieves the row count for a given table.

        Args:
            table (str): The name of the table for which to get the row count.

        Returns:
            int: The number of rows in the table if successful, None if an error occurs.
        """
        try:
            self._cursor.execute(f"SELECT COUNT(*) FROM {table};")
            return self._cursor.fetchone()[0]
        except Exception as e:
            print("Error getting table row count:", e)


    def get_table_data(self, table: str) -> np.ndarray | None:
        """
        Retrieves all data from a given table as a NumPy array.

        Args:
            table (str): The name of the table to retrieve data from.

        Returns:
            np.ndarray: A NumPy array containing the data from the table if successful, None if an error occurs.
        """
        try:
            self._cursor.execute(f"SELECT * FROM {table}")
            data = self._cursor.fetchall()
            column_names = self.get_column_names(table)
            df = pd.DataFrame(data, columns=column_names)
            return df.to_numpy()
        except Exception as e:
            print("Error getting table data array:", e)


    def get_table_dataframe(self, table: str) -> pd.DataFrame | None:
        """
        Retrieves all data from a given table as a Pandas DataFrame.

        Args:
            table (str): The name of the table to retrieve data from.

        Returns:
            pd.DataFrame: A DataFrame containing the data from the table if successful, None if an error occurs.
        """
        try:
            self._cursor.execute(f"SELECT * from {table}")
            data = self._cursor.fetchall()
            df = pd.DataFrame.from_records(data)
            df.columns = self.get_column_names(table)
            return df
        except Exception as e:
            print("Error getting table dataframe:", e)


    def get_table_dataframe_nrows(self, table: str, n: int) -> pd.DataFrame | None:
        """
        Retrieves the first `n` rows of a given table as a Pandas DataFrame.

        Args:
            table (str): The name of the table to retrieve data from.
            n (int): The number of rows to retrieve.

        Returns:
            pd.DataFrame: A DataFrame containing the first `n` rows from the table if successful, None if an error occurs.
        """
        try:
            query = "SELECT * from tripdata"
            data = self._cursor.execute(query).fetchmany(n)
            df = pd.DataFrame.from_records(data)
            df.columns = self.get_column_names(table)
            return df
        except Exception as e:
            print(f"Error getting the first {n} rows from table:", e)


    def get_table_dataframe_random_sample(self, table: str, percentage: float = 0.05) -> pd.DataFrame | None:
        """
        Retrieves a random sample of rows from the specified table based on a given percentage.

        This method selects a random subset of the rows from the given table. The percentage of
        rows to sample is specified by the `percentage` argument. The sampling is done using
        the SQLite `RANDOM()` function, which provides a pseudo-random selection of rows.

        Args:
            table (str): The name of the table from which to retrieve the random sample.
            percentage (float, optional): The fraction of rows to sample, between 0.0 and 1.0.
                                          Defaults to 0.05 (5%).

        Returns:
            pd.DataFrame | None: A Pandas DataFrame containing the random sample of rows.
                                  Returns an empty DataFrame if no data is found, or None if an error occurs.

        Raises:
            ValueError: If the `percentage` is not between 0.0 and 1.0.
        """
        try:
            if not 0.0 <= percentage <= 1.0:
                raise ValueError("Percentage must be between 0.0 and 1.0")

            size = self.get_table_row_count(table)
            threshold = int(round(percentage * size))

            query = "SELECT * FROM tripdata WHERE ABS(RANDOM()) % ? < ?"

            self._cursor.execute(query, (size, threshold))
            data = self._cursor.fetchall()

            if not data:
                return pd.DataFrame(columns=self.get_column_names(table))

            df = pd.DataFrame.from_records(data)
            df.columns = self.get_column_names(table)
            return df
        except Exception as e:
            print(f"Error getting random sample from table:", e)

In [25]:
class DataAnalizer:
    """
    A class to load and split a dataset into training and testing sets.

    This class is designed to handle the process of dividing a DataFrame into
    training and testing sets for machine learning tasks. It takes in a DataFrame,
    a target column, and splits the data into features and labels, which are then
    separated into training and testing datasets.

    Attributes:
        df (pd.DataFrame): The DataFrame containing the data.
        target (str): The name of the target column in the dataset.
        test_size (float): The proportion of the dataset to include in the test split (default is 0.2).
        random_state (int or None): The random seed for reproducibility of the split (default is None).
        data_train (pd.DataFrame): The feature data for the training set.
        labels_train (pd.Series): The labels for the training set.
        data_test (pd.DataFrame): The feature data for the testing set.
        labels_test (pd.Series): The labels for the testing set.
    """

    def __init__(self, df: pd.DataFrame, target: str,  test_size= 0.2, random_state=None):
        """
        Initializes the DataLoader instance and splits the dataset into training and test sets.

        Args:
            df (pd.DataFrame): The dataset to be split.
            target (str): The name of the target column in the dataset.
            test_size (float, optional): The proportion of the dataset to include in the test split. Default is 0.2.
            random_state (int or None, optional): The random seed used for shuffling the data. Default is None.
        """
        self.df = df
        self.target = target
        self.test_size = test_size
        self.random_state = random_state
        self.data_train = None
        self.labels_train = None
        self.data_test = None
        self.labels_test = None

        #divide data in train and test sets
        self._divide_data()


    def _divide_data(self):
        """
        Splits the DataFrame into features (X) and labels (y), and divides them into training and testing sets.

        The method uses `train_test_split()` from scikit-learn to randomly split the data into
        training and test sets based on the specified `test_size` and `random_state`.

        The method sets the following attributes:
            - data_train: Feature data for training.
            - labels_train: Labels for training.
            - data_test: Feature data for testing.
            - labels_test: Labels for testing.
        """
        x = self.df.drop(columns=[self.target])
        y = self.df[self.target]
        x_train, y_train, x_test, y_test = train_test_split(
            x, y, test_size=self.test_size,random_state=self.random_state
        )
        self.data_train  = x_train
        self.labels_train = y_train
        self.data_test    = x_test
        self.labels_test  = y_test
        print("Data divided successfully.")


    def _drop_columns(self, cols):
        """
        Drops a specified feature columns from both the training and testing datasets.

        Args:
            cols (str): The list of the columns to be dropped from both the training and testing datasets.
        """
        try:
            self.data_train.drop(cols, axis=1, inplace=True)
            self.data_test.drop(cols, axis=1, inplace=True)
            print(f"Feature columns {cols} dropped successfully.")
        except Exception as e:
            print(f"Error dropping columns {cols}:", e)



class TripDataAnalizer(DataAnalizer):
    def __init__(self, df: pd.DataFrame, target: str, test_size=0.2, random_state=None):
        super().__init__(df, target, test_size, random_state)


    # Metodo para separar "tpep_pickup_datetime" e "tpep_dropoff_datetime" em várias features
    def extract_datetime_features(self):
        # Ensure datetime columns are in the correct format
        self.df['tpep_pickup_datetime'] = pd.to_datetime(self.df['tpep_pickup_datetime'])
        self.df['tpep_dropoff_datetime'] = pd.to_datetime(self.df['tpep_dropoff_datetime'])

        print(self.df[['tpep_pickup_datetime', 'tpep_dropoff_datetime']].head())

        # Extract features from pickup time
        self.df['pickup_hour'] = self.df['tpep_pickup_datetime'].dt.hour
        self.df['pickup_day_of_week'] = self.df['tpep_pickup_datetime'].dt.dayofweek
        self.df['pickup_month'] = self.df['tpep_pickup_datetime'].dt.month

        # Extract features from dropoff time
        self.df['dropoff_hour'] = self.df['tpep_dropoff_datetime'].dt.hour
        self.df['dropoff_day_of_week'] = self.df['tpep_dropoff_datetime'].dt.dayofweek
        self.df['dropoff_month'] = self.df['tpep_dropoff_datetime'].dt.month

        # Compute trip duration in minutes
        self.df['trip_duration_min'] = (self.df['tpep_dropoff_datetime'] - self.df['tpep_pickup_datetime']).dt.total_seconds() / 60

        print("Feature columns extracted successfully.")
        return self.df

    # Metodo para calcular o tempo de duração da viagem
    def calculate_trip_duration(self):
        """Calculates the duration of the trip in minutes."""

        # Compute trip duration in minutes
        self.df['trip_duration_min'] = (self.df['tpep_dropoff_datetime'] - self.df['tpep_pickup_datetime']).dt.total_seconds() / 60

        print("Trip duration calculated successfully.")
        return self.df

    # Metodo para calcular a velocidade média da viagem
    def calculate_average_speed(self):
        """Calculates the average speed of the trip in km/h."""

        # Ensure the dataset has the necessary distance column
        if 'trip_distance' not in self.df.columns:
            raise ValueError("Column 'trip_distance' is missing from the dataset.")

        # Avoid division by zero for extremely short trips
        self.df['average_speed_kmh'] = self.df['trip_distance'] / (self.df['trip_duration_min'] / 60)
        self.df['average_speed_kmh'].fillna(0, inplace=True)  # Replace NaN values with 0

        print("Average speed calculated successfully.")
        return self.df




Data divided successfully.
  tpep_pickup_datetime tpep_dropoff_datetime  trip_distance
0  2024-03-01 10:30:00   2024-03-01 10:50:00            5.2
1  2024-03-01 11:15:00   2024-03-01 11:45:00            8.5
  tpep_pickup_datetime tpep_dropoff_datetime
0  2024-03-01 10:30:00   2024-03-01 10:50:00
1  2024-03-01 11:15:00   2024-03-01 11:45:00
Feature columns extracted successfully.
  tpep_pickup_datetime tpep_dropoff_datetime  trip_distance  pickup_hour  \
0  2024-03-01 10:30:00   2024-03-01 10:50:00            5.2           10   
1  2024-03-01 11:15:00   2024-03-01 11:45:00            8.5           11   

   pickup_day_of_week  pickup_month  dropoff_hour  dropoff_day_of_week  \
0                   4             3            10                    4   
1                   4             3            11                    4   

   dropoff_month  trip_duration_min  
0              3               20.0  
1              3               30.0  
Average speed calculated successfully.
  tpep_pickup

C:\Users\bruno\AppData\Local\Temp\ipykernel_20988\1604569215.py:136: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  self.df['average_speed_kmh'].fillna(0, inplace=True)  # Replace NaN values with 0


In [19]:
# Data Preprocessing


Hello


In [ ]:
# Data Cleaning

In [17]:
# Testing implementation

# as tabelas vão de "\\2019\\2019-01.sqlite" a "\\2019\\2019-12.sqlite"

sql_loader = KagglehubSQLiteLoader("dhruvildave/new-york-city-taxi-trips-2019", "\\2019\\2019-01.sqlite")
df = sql_loader.get_table_dataframe_nrows("tripdata", 10)
df


Dataset downloaded Successfully 
Dataset saved on:  C:\Users\bruno\.cache\kagglehub\datasets\dhruvildave\new-york-city-taxi-trips-2019\versions\4


,vendorid,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,ratecodeid,store_and_fwd_flag,pulocationid,dolocationid,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1.0,2019-01-01 00:46:40.000000,2019-01-01 00:53:20.000000,1.0,1.5,1.0,N,151.0,239.0,1.0,7.0,0.5,0.5,1.65,0.00,0.3,9.95,None
1,1.0,2019-01-01 00:59:47.000000,2019-01-01 01:18:59.000000,1.0,2.6,1.0,N,239.0,246.0,1.0,14.0,0.5,0.5,1.00,0.00,0.3,16.30,None
2,2.0,2018-12-21 13:48:30.000000,2018-12-21 13:52:40.000000,3.0,0.0,1.0,N,236.0,236.0,1.0,4.5,0.5,0.5,0.00,0.00,0.3,5.80,None
3,2.0,2018-11-28 15:52:25.000000,2018-11-28 15:55:45.000000,5.0,0.0,1.0,N,193.0,193.0,2.0,3.5,0.5,0.5,0.00,0.00,0.3,7.55,None
4,2.0,2018-11-28 15:56:57.000000,2018-11-28 15:58:33.000000,5.0,0.0,2.0,N,193.0,193.0,2.0,52.0,0.0,0.5,0.00,0.00,0.3,55.55,None
5,2.0,2018-11-28 16:25:49.000000,2018-11-28 16:28:26.000000,5.0,0.0,1.0,N,193.0,193.0,2.0,3.5,0.5,0.5,0.00,5.76,0.3,13.31,None
6,2.0,2018-11-28 16:29:37.000000,2018-11-28 16:33:43.000000,5.0,0.0,2.0,N,193.0,193.0,2.0,52.0,0.0,0.5,0.00,0.00,0.3,55.55,None
7,1.0,2019-01-01 00:21:28.000000,2019-01-01 00:28:37.000000,1.0,1.3,1.0,N,163.0,229.0,1.0,6.5,0.5,0.5,1.25,0.00,0.3,9.05,None
8,1.0,2019-01-01 00:32:01.000000,2019-01-01 00:45:39.000000,1.0,3.7,1.0,N,229.0,7.0,1.0,13.5,0.5,0.5,3.70,0.00,0.3,18.50,None
9,1.0,2019-01-01 00:57:32.000000,2019-01-01 01:09:32.000000,2.0,2.1,1.0,N,141.0,234.0,1.0,10.0,0.5,0.5,1.70,0.00,0.3,13.00,None


# EDA (Exploratory Data Analysis)

In [2]:
# Libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3 as sql
import kagglehub

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler


C:\Users\bruno\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Data Analysis

A análise de dados será realizada com a abordagem ELT (Extract, Load, Transform). Inicialmente, os dados serão extraídos do KaggleHub e uma amostra aleatória será utilizada para trabalhar com uma fração representativa do conjunto completo. Em seguida, os dados extraídos serão carregados em estruturas como DataFrames Pandas ou arrays NumPy, facilitando a manipulação e análise. Por fim, na fase de transformação, os dados serão limpos, normalizados e preparados para gerar métricas e insights. A amostragem aleatória permite reduzir a complexidade, permitindo uma análise eficiente sem processar todos os dados de uma vez.

In [3]:
class KagglehubDatabaseLoader:
    """
    A class to download and manage datasets from KaggleHub.

    This class automates the process of downloading a dataset from KaggleHub
    and provides a method to retrieve the local file path.

    Attributes:
        _dataset (str): The name of the dataset to be downloaded.
        _path (str or None): The local directory path where the dataset is saved.
    """

    def __init__(self, dataset: str):
        """
        Initializes the KagglehubDatabaseLoader class.

        Args:
            dataset (str): The name of the dataset to be downloaded.
        """
        self._dataset = dataset
        self._path = None

        self._download_dataset()


    def _download_dataset(self) -> None:
        """
        Downloads the specified dataset from KaggleHub.

        This method attempts to download the dataset and assigns the local
        file path to the `_path` attribute. If an error occurs during the
        download, an exception is caught and printed.
        """
        try:
            self._path = kagglehub.dataset_download(self._dataset)
            print("Dataset downloaded Successfully ")
            print("Dataset saved on: ", self._path)
        except Exception as e:
            print("Error downloading dataset:", e)


    def get_path(self) -> str | None:
        """
        Returns the local directory path of the downloaded dataset.

        Returns:
            str or None: The file path of the dataset if the download was
                        successful, otherwise None.
        """
        return self._path


class KagglehubSQLiteLoader(KagglehubDatabaseLoader):
    """
    A subclass of KagglehubDatabaseLoader to interact with an SQLite dataset.

    This class adds methods to retrieve information about tables, numpy arrays and dataframes.
    the SQLite dataset is downloaded from KaggleHub.

    Attributes:
        _dataset (str): The name of the dataset to be downloaded from KaggleHub.
        _path (str or None): The local directory path where the dataset is saved.
        _file_path (str or None): The complete path to the specific SQLite file within the dataset.
        _conn: The SQLite connection object used to interact with the database.
        _cursor: The cursor object for executing SQL queries on the database.
    """

    def __init__(self, dataset: str, file: str):
        """
        Initializes the KagglehubSQLLoader class by calling the superclass constructor
        to download the dataset and setting up the SQLite connection.

        Args:
            dataset (str): The dataset name to be downloaded.
            file (str): The name of the SQLite file in the downloaded dataset to be opened.
        """
        super().__init__(dataset)
        self._file_path = self._path + file
        self._conn = sql.connect(self._file_path)
        self._cursor = self._conn.cursor()


    def get_table_names(self) -> list | None:
        """
        Retrieves all table names from the SQLite database.

        Returns:
            list: A list of table names if successful, None if an error occurs.
        """
        try:
            table_name_query = "SELECT name FROM sqlite_master WHERE type='table';"
            return self._cursor.execute(table_name_query).fetchall()
        except Exception as e:
            print("Error getting table names:", e)


    def get_column_names(self, table: str) -> list | None:
        """
        Retrieves the column names for a given table in the SQLite database.

        Args:
            table (str): The name of the table for which to get column names.

        Returns:
            list: A list of column names if successful, None if an error occurs.
        """
        try:
            self._cursor.execute(f"PRAGMA table_info({table});")
            columns = [col[1] for col in self._cursor.fetchall()]
            return columns
        except Exception as e:
            print("Error getting column names:", e)


    def get_table_row_count(self, table: str) -> int | None:
        """
        Retrieves the row count for a given table.

        Args:
            table (str): The name of the table for which to get the row count.

        Returns:
            int: The number of rows in the table if successful, None if an error occurs.
        """
        try:
            self._cursor.execute(f"SELECT COUNT(*) FROM {table};")
            return self._cursor.fetchone()[0]
        except Exception as e:
            print("Error getting table row count:", e)


    def get_table_data(self, table: str) -> np.ndarray | None:
        """
        Retrieves all data from a given table as a NumPy array.

        Args:
            table (str): The name of the table to retrieve data from.

        Returns:
            np.ndarray: A NumPy array containing the data from the table if successful, None if an error occurs.
        """
        try:
            self._cursor.execute(f"SELECT * FROM {table}")
            data = self._cursor.fetchall()
            column_names = self.get_column_names(table)
            df = pd.DataFrame(data, columns=column_names)
            return df.to_numpy()
        except Exception as e:
            print("Error getting table data array:", e)


    def get_table_dataframe(self, table: str) -> pd.DataFrame | None:
        """
        Retrieves all data from a given table as a Pandas DataFrame.

        Args:
            table (str): The name of the table to retrieve data from.

        Returns:
            pd.DataFrame: A DataFrame containing the data from the table if successful, None if an error occurs.
        """
        try:
            self._cursor.execute(f"SELECT * from {table}")
            data = self._cursor.fetchall()
            df = pd.DataFrame.from_records(data)
            df.columns = self.get_column_names(table)
            return df
        except Exception as e:
            print("Error getting table dataframe:", e)


    def get_table_dataframe_nrows(self, table: str, n: int) -> pd.DataFrame | None:
        """
        Retrieves the first `n` rows of a given table as a Pandas DataFrame.

        Args:
            table (str): The name of the table to retrieve data from.
            n (int): The number of rows to retrieve.

        Returns:
            pd.DataFrame: A DataFrame containing the first `n` rows from the table if successful, None if an error occurs.
        """
        try:
            query = "SELECT * from tripdata"
            data = self._cursor.execute(query).fetchmany(n)
            df = pd.DataFrame.from_records(data)
            df.columns = self.get_column_names(table)
            return df
        except Exception as e:
            print(f"Error getting the first {n} rows from table:", e)


    def get_table_dataframe_random_sample(self, table: str, percentage: float = 0.05) -> pd.DataFrame | None:
        """
        Retrieves a random sample of rows from the specified table based on a given percentage.

        This method selects a random subset of the rows from the given table. The percentage of
        rows to sample is specified by the `percentage` argument. The sampling is done using
        the SQLite `RANDOM()` function, which provides a pseudo-random selection of rows.

        Args:
            table (str): The name of the table from which to retrieve the random sample.
            percentage (float, optional): The fraction of rows to sample, between 0.0 and 1.0.
                                          Defaults to 0.05 (5%).

        Returns:
            pd.DataFrame | None: A Pandas DataFrame containing the random sample of rows.
                                  Returns an empty DataFrame if no data is found, or None if an error occurs.

        Raises:
            ValueError: If the `percentage` is not between 0.0 and 1.0.
        """
        try:
            if not 0.0 <= percentage <= 1.0:
                raise ValueError("Percentage must be between 0.0 and 1.0")

            size = self.get_table_row_count(table)
            threshold = int(round(percentage * size))

            query = "SELECT * FROM tripdata WHERE ABS(RANDOM()) % ? < ?"

            self._cursor.execute(query, (size, threshold))
            data = self._cursor.fetchall()

            if not data:
                return pd.DataFrame(columns=self.get_column_names(table))

            df = pd.DataFrame.from_records(data)
            df.columns = self.get_column_names(table)
            return df
        except Exception as e:
            print(f"Error getting random sample from table:", e)

In [14]:
class DataAnalizer:
    """
    A class to load and split a dataset into training and testing sets.

    This class is designed to handle the process of dividing a DataFrame into
    training and testing sets for machine learning tasks. It takes in a DataFrame,
    a target column, and splits the data into features and labels, which are then
    separated into training and testing datasets.

    Attributes:
        df (pd.DataFrame): The DataFrame containing the data.
        target (str): The name of the target column in the dataset.
        test_size (float): The proportion of the dataset to include in the test split (default is 0.2).
        random_state (int or None): The random seed for reproducibility of the split (default is None).
        data_train (pd.DataFrame): The feature data for the training set.
        labels_train (pd.Series): The labels for the training set.
        data_test (pd.DataFrame): The feature data for the testing set.
        labels_test (pd.Series): The labels for the testing set.
    """

    def __init__(self, df: pd.DataFrame, target: str,  test_size= 0.2, random_state=None):
        """
        Initializes the DataLoader instance and splits the dataset into training and test sets.

        Args:
            df (pd.DataFrame): The dataset to be split.
            target (str): The name of the target column in the dataset.
            test_size (float, optional): The proportion of the dataset to include in the test split. Default is 0.2.
            random_state (int or None, optional): The random seed used for shuffling the data. Default is None.
        """
        self.df = df
        self.target = target
        self.test_size = test_size
        self.random_state = random_state
        self.data_train = None
        self.labels_train = None
        self.data_test = None
        self.labels_test = None

        #divide data in train and test sets
        self._divide_data()


    def _divide_data(self):
        """
        Splits the DataFrame into features (X) and labels (y), and divides them into training and testing sets.

        The method uses `train_test_split()` from scikit-learn to randomly split the data into
        training and test sets based on the specified `test_size` and `random_state`.

        The method sets the following attributes:
            - data_train: Feature data for training.
            - labels_train: Labels for training.
            - data_test: Feature data for testing.
            - labels_test: Labels for testing.
        """
        x = self.df.drop(columns=[self.target])
        y = self.df[self.target]
        x_train, y_train, x_test, y_test = train_test_split(
            x, y, test_size=self.test_size,random_state=self.random_state
        )
        self.data_train  = x_train
        self.labels_train = y_train
        self.data_test    = x_test
        self.labels_test  = y_test
        print("Data divided successfully.")


    def _drop_columns(self, cols):
        """
        Drops a specified feature columns from both the training and testing datasets.

        Args:
            cols (str): The list of the columns to be dropped from both the training and testing datasets.
        """
        try:
            self.data_train.drop(cols, axis=1, inplace=True)
            self.data_test.drop(cols, axis=1, inplace=True)
            print(f"Feature columns {cols} dropped successfully.")
        except Exception as e:
            print(f"Error dropping columns {cols}:", e)


class TripDataAnalizer(DataAnalizer):
    def __init__(self, df: pd.DataFrame, target: str, test_size=0.2, random_state=None):
        super().__init__(df, target, test_size, random_state)


    # Metodo para separar "tpep_pickup_datetime" e "tpep_dropoff_datetime" em várias features


    # Metodo para calcular o tempo de duração da viagem

    # Metodo para calcular a velocidade média da viagem

In [ ]:
# Data Preprocessing

In [ ]:
# Data Cleaning

In [4]:
# Testing implementation

# as tabelas vão de "\\2019\\2019-01.sqlite" a "\\2019\\2019-12.sqlite"

sql_loader = KagglehubSQLiteLoader("dhruvildave/new-york-city-taxi-trips-2019", "\\2019\\2019-01.sqlite")
df = sql_loader.get_table_dataframe_nrows("tripdata", 1000)
df


100%|██████████| 2.38G/2.38G [01:02<00:00, 40.8MB/s]

Extracting files...


Error downloading dataset: [Errno 28] No space left on device


TypeError: unsupported operand type(s) for +: 'NoneType' and 'str'

# EDA (Exploratory Data Analysis)